# Collecting Geo Data for Kansas City

In [1]:
#importing the libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

import re #regular expessions

! pip install beautifulsoup4
! pip install requests
import requests #request online data
from bs4 import BeautifulSoup as bs # library for navigating html files

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.52         |     pyhd8ed1ab_0          35 KB  conda-forge
    geopy-2.2.0                |     pyhd8ed1ab_0          67 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         102 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.52-pyhd8ed1ab_0
  geopy              conda-forge/noarch::geopy-2.2.0-pyhd8ed1ab_0



geographiclib-1.52   | 35 KB     | ##################################### | 100% 
geopy-2.2.0          | 67 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transa

In [2]:
KC_df = pd.read_csv('kansas_city_data.csv')
KC_df.head()

,Area,Neighborhood
0,CBD-Downtown,CBD-Downtown
1,Greater Downtown,18th and Vine
2,Greater Downtown,Beacon Hill-McFeders
3,Greater Downtown,Columbus Park
4,Greater Downtown,Crossroads


In [2]:
CLIENT_ID = '4TJ1SLTDO1KADL553GXCDXOJXZONZYTBIRQMVBQ32JYAU3EA' # your Foursquare ID
CLIENT_SECRET = 'TC1N45PW5LSGVLIZ02DDPA2JOMWB2I3BA33OPX55VW2LBKBF' # your Foursquare Secret
ACCESS_TOKEN = 'NS3WWURVAWWAGWSTUXQDQCS0150IPNQIYNHVE5JM1OSX2CAV' # your FourSquare Access Token

In [1]:
VERSION = '20180604'
LIMIT = 30

In [4]:
hood_longitude_list = list()
hood_latitude_list = list()
for neighborhood in KC_df['Neighborhood']:
    try:
        address = '{}, Kansas City, MO'.format(neighborhood)
        geolocator = Nominatim(user_agent="foursquare_agent")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        hood_longitude_list.append(longitude)
        hood_latitude_list.append(latitude)
    except:
        latitude = 'No Data Found'
        longitude = 'No Data Found'
        hood_longitude_list.append(longitude)
        hood_latitude_list.append(latitude)
print('hoods printed')

hoods printed


In [5]:
area_longitude_list = list()
area_latitude_list = list()
for area in KC_df['Area']:
    try:
        address = '{}, Kansas City, MO'.format(area)
        geolocator = Nominatim(user_agent="foursquare_agent")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        area_longitude_list.append(longitude)
        area_latitude_list.append(latitude)
    except:
        latitude = 'No Data Found'
        longitude = 'No Data Found'
        area_longitude_list.append(longitude)
        area_latitude_list.append(latitude)
print('areas printed')

areas printed


In [6]:
KC_df.insert(loc=2,
    column='area_longitude',
    value=area_longitude_list
)
KC_df.insert(loc=3,
    column='area_latitude',
    value=area_latitude_list
)
KC_df.insert(loc=4,
    column='hood_longitude',
    value=hood_longitude_list
)
KC_df.insert(loc=5,
    column='hood_latitude',
    value=hood_latitude_list
)
KC_df.head()

,Area,Neighborhood,area_longitude,area_latitude,hood_longitude,hood_latitude
0,CBD-Downtown,CBD-Downtown,-94.5788,39.0917,-94.5788,39.0917
1,Greater Downtown,18th and Vine,No Data Found,No Data Found,-94.5539,39.0944
2,Greater Downtown,Beacon Hill-McFeders,No Data Found,No Data Found,No Data Found,No Data Found
3,Greater Downtown,Columbus Park,No Data Found,No Data Found,-94.5704,39.1104
4,Greater Downtown,Crossroads,No Data Found,No Data Found,-94.5837,39.0901


In [7]:
#to prove df is storing correct values and not rounding them
for hoodlat in KC_df['hood_latitude'][0:5]:
    print(hoodlat)

39.0917491
39.09444044999999
No Data Found
39.110399
39.0901009


In [8]:
KC_df = KC_df.set_index('Area')

In [12]:
KC_df.to_csv('Kansas_City_geodata.csv')

## Visualizing Neighborhoods

In [4]:
map_kansascity = folium.Map(location=[latitude, longitude], zoom_start=10)
count= -1
counting_errors = 0
# add markers to map
for lat, lng, neighborhood in zip(KC_df['hood_latitude'], KC_df['hood_longitude'], KC_df['Neighborhood']):
    try:
        count = count +1
        label = '{}'.format(neighborhood)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_kansascity)
    except:
        count = count + 1
        counting_errors = counting_errors + 1
        print('no neighborhood to label.', 'index:',count)
print(counting_errors)

count = -1
counting_errors = 0
for lat, lng, area in zip(KC_df['area_latitude'], KC_df['area_longitude'],KC_df['Area']):
    try:
        count = count +1
        label = '{}'.format(area)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=1.0,
            parse_html=False).add_to(map_kansascity)
    except:
        count = count + 1
        counting_errors = counting_errors + 1
        print('no area to label.', 'index:',count)
map_kansascity

NameError: name 'latitude' is not defined